In [14]:
import re

import pandas as pd

from transformers import AutoModel, AutoTokenizer

In [2]:
MODEL_CHECKPOINT = 'EleutherAI/gpt-j-6b'
MODEL_CHECKPOINT = 'ainize/gpt-j-6B-float16'
MODEL_CHECKPOINT = 'ainize/kobart-news'
MODEL_CHECKPOINT = 'paust/pko-t5-base'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
DATA_PATH = 'data/20230412_50000_from_db.pickle'
DATA_PATH = 'data/20230413_300000_from_db.pickle'

df = pd.read_pickle(DATA_PATH)

In [5]:
### 제목 혹은 본문이 nan값인 경우 제외

checker = df.title.isna().apply(lambda x: not x)
df = df[checker]
checker = df.content.isna().apply(lambda x: not x)
df = df[checker]

In [6]:
### 본문 내용이 없는 경우 제외

checker = df.content.apply(lambda x: x != 'None')
df = df[checker]

In [7]:
### 제목이 중복된 경우 같은 샘플로 취급하여 첫번째 것만 남기고 제외

checker = df.title.duplicated().apply(lambda x: not x)
df = df[checker]

In [8]:
### new-line 공백으로 교체

df['title'] = df.title.str.replace('\n', ' ')
df['content'] = df.content.str.replace('\n', ' ')

In [9]:
### 문자열 스트립

df['title'] = df.title.apply(lambda x: x.strip())
df['content'] = df.content.apply(lambda x: x.strip())

In [10]:
### 모델의 입력이 될 제목과 기사를 공백으로 묶어 title_content 칼럼 생성

df['title_content'] = df.title + ' ' + df.content

In [11]:
### 연속 공백을 공백 하나로 치환

regex_pat = re.compile(r'[\s]+')

df['title_content'] = df.title_content.str.replace(regex_pat, ' ')

In [12]:
df = df[['id', 'create_date', 'title', 'title_content']]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248300 entries, 0 to 299999
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             248300 non-null  int64         
 1   create_date    248300 non-null  datetime64[ns]
 2   title          248300 non-null  object        
 3   title_content  248300 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 9.5+ MB


In [23]:
### Check English only articles

# regex_pat = re.compile(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+')

# checker = df.title.str.contains(regex_pat).apply(lambda x: not x)
# df[checker]

In [13]:
df['len_tokenized'] = df.title_content.apply(lambda x: len(tokenizer(x)['input_ids']))

### Tokenized Sequence Length Shorter Than 1000 Only

In [14]:
checker = df.len_tokenized < 1000
df = df[checker]
df.len_tokenized.describe()

count    193282.000000
mean        510.867360
std         262.128131
min           5.000000
25%         289.000000
50%         518.000000
75%         723.000000
max         999.000000
Name: len_tokenized, dtype: float64

### Tokenized Sequence Length Longer Than 250 Only

In [15]:
# checker = df.len_tokenized > 250
# temp_df = df[checker]
# temp_df.info(), temp_df.len_tokenized.describe()

In [16]:
# temp = 0
# for idx, row in temp_df.iterrows():
#     if temp == 10000:
#         break
#     print(row.title_content)
#     print()
#     temp += 1

### Tokenized Sequence Length Longer Than 500 Only

In [17]:
checker = df.len_tokenized > 500
temp_df = df[checker]
temp_df.info(), temp_df.len_tokenized.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100886 entries, 2 to 299998
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             100886 non-null  int64         
 1   create_date    100886 non-null  datetime64[ns]
 2   title          100886 non-null  object        
 3   title_content  100886 non-null  object        
 4   len_tokenized  100886 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 4.6+ MB


(None,
 count    100886.000000
 mean        724.926808
 std         139.606260
 min         501.000000
 25%         604.000000
 50%         713.000000
 75%         839.000000
 max         999.000000
 Name: len_tokenized, dtype: float64)

In [18]:
# temp = 0
# for idx, row in temp_df.iterrows():
#     if temp == 10000:
#         break
#     print(row.title_content)
#     print()
#     temp += 1

In [19]:
temp_df

,id,create_date,title,title_content,len_tokenized
2,12846519,2023-04-13 00:53:38,"여자테니스 김다빈·백다연, 국가대항전 빌리진킹컵 중국에 완승","여자테니스 김다빈·백다연, 국가대항전 빌리진킹컵 중국에 완승 국가대항전 빌리진킹컵,...",602
3,12846514,2023-04-13 00:51:00,"전진 ""혼수상태에서 깨어난 후 '드림팀' 출연""...美친 승부욕 (라스)[종합]","전진 ""혼수상태에서 깨어난 후 '드림팀' 출연""...美친 승부욕 (라스)[종합] 이...",836
4,12846515,2023-04-13 00:50:00,"""김호중 형이랑 같은 생각""..TAN, '너목보10' 리액션 장인들","""김호중 형이랑 같은 생각""..TAN, '너목보10' 리액션 장인들 이미지 원본보기...",515
6,12846516,2023-04-13 00:48:00,‘유퀴즈’ 김희애 “껍데기만 남아있는 느낌…이제야 좋은 인간 된 것 같아”,‘유퀴즈’ 김희애 “껍데기만 남아있는 느낌…이제야 좋은 인간 된 것 같아” 배우 김...,684
9,12846511,2023-04-13 00:44:22,"美공화 유일한 흑인 상원의원 대선 도전…""바이든 실패에 초점""","美공화 유일한 흑인 상원의원 대선 도전…""바이든 실패에 초점"" 인종차별 트럼프에 '...",872
...,...,...,...,...,...
299993,12424414,2023-03-15 11:00:00,"최윤영, 러블리 삐약이 모습 어디로?…안하무인 상속녀로 첫 등장 ('비밀의 여자')","최윤영, 러블리 삐약이 모습 어디로?…안하무인 상속녀로 첫 등장 ('비밀의 여자')...",548
299994,12424413,2023-03-15 11:00:00,김새론 알바 사진 올린 이유? “생활고 어필 위해”,김새론 알바 사진 올린 이유? “생활고 어필 위해” 만취 음주운전 사고 후 생활고를...,815
299996,12424411,2023-03-15 11:00:00,"'더퀸즈' 제작사 이앤엠씨, 올댓마케팅 업무협약","'더퀸즈' 제작사 이앤엠씨, 올댓마케팅 업무협약 ""저변 크게 확대되길 기대""이미지 ...",597
299997,12424410,2023-03-15 11:00:00,"‘나는 솔로‘, ‘일름보 솔로男‘ 등장…“진짜 좋아하면 제거하는 거야”","‘나는 솔로‘, ‘일름보 솔로男‘ 등장…“진짜 좋아하면 제거하는 거야” 이미지 원본...",765


### Shuffling

In [20]:
temp_df = temp_df.sample(frac=1, random_state=100)

### Split DF By 1000 Rows

In [21]:
temp_df.reset_index(drop=True, inplace=True)

In [22]:
raw_dfs = []

start = 0
for idx in range(1000, len(temp_df)+1000, 1000):
    raw_dfs.append(temp_df.iloc[start:idx])
    start = idx

### Save Raw Data

In [23]:
import os

SAVE_PATH = f'./data/{len(temp_df)}_from_300000_seq_len_from_500_to_1000'
os.makedirs(SAVE_PATH, exist_ok=True)

for idx, df in enumerate(raw_dfs):
    file_name = f'{len(temp_df)}_from_300000_seq_len_from_500_to_1000_{idx}'
    # print(f'{SAVE_PATH}/{file_name}.pickle')
    df.to_pickle(f'{SAVE_PATH}/{file_name}.pickle')

In [28]:
SAVE_PATH

'./data/100886_from_300000_seq_len_from_500_to_1000'

### Check Data

In [26]:
def print_samples(df):
    for idx, row in df.iterrows():
        print(row.title)
        print(row.title_content)
        print()

In [33]:
len(raw_dfs)

101

In [34]:
for idx, df in enumerate(raw_dfs[100:]):
    if idx == 10:
        break
    print_samples(df)

수사 시작 1년 반 만에 이재명 기소…"이제 법원의 시간"
수사 시작 1년 반 만에 이재명 기소…"이제 법원의 시간" <앵커> 검찰이 민주당 이재명 대표를 불구속 상태로 재판에 넘겼습니다. 위례신도시와 대장동 비리 의혹 수사가 시작된 지 약 1년 반 만입니다. 검찰은 이 대표를 민간 업자가 막대한 개발 이익을 챙긴 대장동 사업의 직접 설계자이자 최종 책임자로 규정하고 5가지 혐의를 적용했습니다. 다만 천화동인 1호에 이 대표 측 지분이 있다는 이른바 428억 원 약정 의혹은 이번 기소 범위에서 빠졌습니다. 검찰과 이재명 대표 모두 이제 법원의 시간이 시작됐다며 치열한 법정 다툼을 예고했습니다. 소환욱 기자입니다. <기자> 민주당 공개 최고위원회의가 끝날 무렵. 이재명 대표가 마이크를 잡았습니다. 검찰 기소는 각본대로 정해진 것이었다며 놀랄 일이 아니라고 말했습니다. [이재명/민주당 대표 : 검찰이 시간을 지연하고 온갖 압수수색쇼, 체포영장쇼를 벌이면서 시간을 끌고 정치적으로 활용하다가 이제 그 정해진 답대로 기소한 것입니다.] 그러면서 법정에서 진실이 가려질 것이라고 자신했습니다. [이재명/민주당 대표 : 법정에서 진실을 가리기 위해서 최선을 다할 것이고, 결국 명명백백하게 진실이 드러날 것이라고 믿습니다.] 기소 6시간 만에 민주당은 당무위원회를 열어 이 대표가 대표직을 유지할 수 있도록 결정했습니다. 부정부패 혐의로 기소 시 직무를 정지한다는 당헌 80조에서 개정 때부터 방탄 논란을 불러왔던, 정치 탄압이 인정되면 달리할 수 있다는 예외 조항을 적용한 것입니다. [김의겸/민주당 대변인 : 탄압 의도에 대해서 당이 단결하고 단합하는 모습을 신속히 보여주는 것이 필요하다는 데 의견을 모았습니다.] 충분한 토론 없이 당무위를 연 것은 무책임하다, 국민들의 반감이 축적될 것이라는 우려가 비명계 의원들 사이에서는 나왔습니다. 국민의힘은 범죄 혐의가 명백한데도 민주당이 방탄 갑옷만 덧대고 있다고 비판했습니다. [김기현/국민의힘 대표 : 매우 심각한 내용인 것 같습니다